In [ ]:
import torch
import torch.nn as nn
from torch.distributions import Categorical
import numpy as np
from RL import ActorCriticAgent
from single_agent_env import SinglePlayerFootball, ACTION_SPACE_SIZE, STATE_SPACE_SIZE
torch.manual_seed(3407)
torch.cuda.manual_seed(3407)
np.random.seed(3407)

In [ ]:
class AC(nn.Module):

    def __init__(self, observation_size, action_size):
        super().__init__()
        self.actor = nn.Sequential(
            nn.Linear(observation_size, 1024),
            nn.LeakyReLU(),
            nn.Linear(1024, 512),
            nn.LeakyReLU(),
            nn.Linear(512, 256),
            nn.LeakyReLU(),
            nn.Linear(256, 128),
            nn.LeakyReLU(),
            nn.Linear(128, 64),
            nn.LeakyReLU(),
            nn.Linear(64, action_size),
            nn.Softmax(dim=0)
        )
        self.critic = nn.Sequential(
            nn.Linear(observation_size, 1024),
            nn.LeakyReLU(),
            nn.Linear(1024, 512),
            nn.LeakyReLU(),
            nn.Linear(512, 256),
            nn.LeakyReLU(),
            nn.Linear(256, 128),
            nn.LeakyReLU(),
            nn.Linear(128, 64),
            nn.LeakyReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        probs = self.actor(x)
        distribution = Categorical(probs)
        action = distribution.sample()

        value = self.critic(x)
        return action.item(), distribution.log_prob(action), value

In [ ]:
agent = ActorCriticAgent(STATE_SPACE_SIZE, ACTION_SPACE_SIZE, device="cuda:0")
agent.create_model(AC, lr=0.0001, y=0.99)

In [ ]:
train_id = "ac_fixed_ball_every_-1_goal_300"
env = SinglePlayerFootball(title=train_id)
best_score = 0
while env.running:
    state = env.reset()
    while not env.loop_once():
        action = agent.policy(state)
        n_state, reward, done = env.step(action)
        agent.learn(state, action, n_state, reward, done)
        state = n_state
        if agent.reward_history[-1] > best_score:
            best_score = agent.reward_history[-1]
            model_scripted = torch.jit.script(agent.model)
            model_scripted.save(f"best_models/{train_id}_{agent.episode_count}_{round(best_score, 6)}.pt")
del env

In [ ]:
with open('rewards.txt', 'w') as f:
    f.writelines([f"{round(item, 6)}\n" for item in agent.reward_history])

In [ ]:
agent.train = False
env = SinglePlayerFootball(title=train_id)
for _ in range(10):
    s = env.reset()
    while not env.loop_once():
        s, _, _ = env.step(agent.policy(s))
del env